In [ ]:
#Sorry for this crazy import cell... this will be cleaned up

import psycopg2
import nltk
import unicodedata
import pandas as pd
import pprint
import pickle
import re
import os
from nltk.corpus import wordnet
import time

stops = nltk.corpus.stopwords.words('english')
wnl = nltk.WordNetLemmatizer()
nltk.download('averaged_perceptron_tagger')

https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

In [ ]:
#Input your PostGres credentials to connect

dbname = ''
username = ''
host = ''
password = ''

conn = psycopg2.connect('dbname={} user={} host={} password={}'.format(dbname, username, host, password))
cur = conn.cursor()

In [ ]:
cur = conn.cursor()
cur.execute("""
    SELECT * FROM review LIMIT 1000
""")

cols = ['review_id', 'user_id', 'business_id', 'stars', 'review_date', 'review_text', 'useful', 'funny', 'cool']

review_sample = pd.DataFrame(cur.fetchall(), columns=cols)

In [ ]:
review_sample

In [ ]:
print(review_sample.loc[24, 'review_text'])

In [ ]:
def _remove_stops(tokens):
    stops = nltk.corpus.stopwords.words('english')
    neg_stops = [
    'no', 'not', 'nor', 'don\'', 'don\'t', 'ain', 
    'ain\'t', 'aren\'t', 'aren', 'couldn', 'couldn\'t', 
    'didn', 'didn\'t', 'doesn', 'doesn\'t', 'hadn', 
    'hadn\'t', 'hasn', 'hasn\'t', 'haven', 'haven\'t',
    'isn', 'isn\'t', 'mightn', 'mightn\'t', 'mustn', 
    'mustn\'t', 'needn', 'needn\'t', 'shan', 'shan\'t',
    'shouldn', 'shouldn\'t', 'wasn', 'wasn\'t', 'weren',
    'weren\'t', 'won', 'won\'t', 'wouldn', 'wouldn\'t'
    ]
#still leaves in but and don.. fix this.. 
#doesn't get rid of other obvious stopwords, like i'm, they're....
    for x in neg_stops:
        if x in stops:
            stops.remove(x)
            
    tokens_without_stops = [token for token in tokens if token not in stops]
    return tokens_without_stops

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

#I think I adjusted this properly to remove digits
#however for phrases that are hyphenated, like chilled-to-the-bone, the code returns the token 'chilledtothebone.'
#This will need to be fixed
def _clean_review(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
    text = re.sub(r"[^A-Za-z\s']", '', text)  
    tokens = text.split()
    for i, token in enumerate(tokens):
        tokens[i] = wnl.lemmatize(token, pos= get_wordnet_pos(token))
    return tokens

def _process_review(text):
    tokens = _remove_stops(_clean_review(text))
    return tokens

In [ ]:
_process_review(review_sample.loc[24, 'review_text'])

In [ ]:
def apply_on_column(data):
    data['review_text'] = data['review_text'].apply(lambda x: _process_review(x))
    return data

In [ ]:
start = time.time()
apply_on_column(review_sample)
end = time.time()
dur = end - start
# Verify that the function is working
print('Processed {} instances in {} minutes {} seconds.\n'.format(review_sample.shape[0], dur//60, dur%60))

In [ ]:
review_sample.loc[27, 'review_text']